In [31]:
import numpy as np 
import pandas as pd 
import re   # 정규표현식

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

PassengerId = test['PassengerId']

train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [7]:
train['Ticket_type'] = train['Ticket'].apply(lambda x : x[0:3])
# ticket 앞에서 3자리만 잘라오기
# ??? 

In [12]:
train['Ticket_type'] = train['Ticket_type'].astype('category')
# object > category(범주형)로 타입 변형

In [14]:
train['Ticket_type'] = train['Ticket_type'].cat.codes
# Series.cat.codes 
# 색인 + 일련의 코드 반환 

In [17]:
test['Ticket_type'] = test['Ticket'].apply(lambda x : x[0:3])
test['Ticket_type'] = test['Ticket_type'].astype('category')
test['Ticket_type'] = test['Ticket_type'].cat.codes


In [22]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_type,Words_Count
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,124,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,137,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,148,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3,7
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,97,4


In [32]:
full_data = [train, test]

# name > 이름 길이
train['Words_Count'] = train['Name'].apply(lambda x: len(x.split()))    # split 비우면 '공백'
test['Words_Count'] = test['Name'].apply(lambda x: len(x.split()))

# Cabin > o / x 
train['Has_Cabin'] = train['Cabin'].apply(lambda x: 0 if type(x) == float else 1 )  # Nan = 실수 
test['Has_Cabin'] = test['Cabin'].apply(lambda x: 0 if type(x) == float else 1 ) 

# SibSp + Parch > FamilySize 
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1     # 본인 포함

# FamilSize > IsAlone 
for dataset in full_data:
    dataset['IsAlone'] = 0                                      # 혼승 X
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1      # 행 : 혼자 탑승, 열 : 혼승 X

# Embarked 결측치(최빈값)
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

# Fare 결측치(중앙값) + 카테고리형 자료로 변형 # ?? qcut 
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 6)

# Age > 카테고리형 자료로 변형  # ?? 
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 6)

# Name > title 추출 함수 
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

# Name > Title 
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

# non_common한 title을 'Rare'로 그룹화
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss') # mlle > miss (replace)
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')


<ipython-input-32-487776377807>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list


In [35]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_type,Words_Count,Has_Cabin,FamilySize,IsAlone,CategoricalFare,CategoricalAge,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,124,4,0,2,0,"(-0.001, 7.775]","(13.333, 26.667]",Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,137,7,1,2,0,"(52.369, 512.329]","(26.667, 40.0]",Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,148,3,0,1,1,"(7.775, 8.662]","(13.333, 26.667]",Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,3,7,1,2,0,"(52.369, 512.329]","(26.667, 40.0]",Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,97,4,0,1,1,"(7.775, 8.662]","(26.667, 40.0]",Mr


In [ ]:
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 6)